In [30]:
import asyncio
import aiohttp
from bs4 import BeautifulSoup
import lxml

In [ ]:
async def get_all_html(firm_code):
    async with aiohttp.ClientSession() as session:
        first_page = await price_data(firm_code, 1, session)
        tree = lxml.html.fromstring(first_page)
        href = tree.xpath("//td[@class='pgRR']/a/@href")
        page_num = int(href[0].split("page=")[-1])
        print(page_num)
        result_htmls = await asyncio.gather(*[price_data(firm_code, x, session) for x in range(1, page_num + 1)])
        return result_htmls

async def price_data(firm_code, num, session):
    price_url = 'https://finance.naver.com/item/sise_day.nhn?code=' +firm_code +'&'+'page=' +str(num)
    html = await fetch(session, price_url)
    return html

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()
    
# 선언한 함수 실행

result = await get_all_html('024850')

for index, html in enumerate(result):
    price_tables=pd.read_html(html)
    price_tables=price_tables[0]
    price_tables=price_tables.set_index(price_tables.columns[0])
    price_tables=price_tables[['시가','종가','고가','저가','거래량']]
    filtered_price=price_tables.dropna()
    if index == 0:
        historic_price=filtered_price
    else:
        historic_price=pd.concat([historic_price, filtered_price])
historic_price

In [ ]:
import pandas as pd
import requests

def price_data(firm_code):
    for num in range(1,466) :
        price_url = 'https://finance.naver.com/item/sise_day.nhn?code=' +firm_code +'&'+'page=' +str(num)
        price_page=requests.get(price_url)
        price_tables=pd.read_html(price_page.text)
        price_tables=price_tables[0]
        price_tables=price_tables.set_index(price_tables.columns[0])
        price_tables=price_tables[['시가','종가','고가','저가','거래량']]
        filtered_price=price_tables.dropna()
        if num==1:
            historic_price=filtered_price
        else:
            historic_price=pd.concat([ historic_price, filtered_price])
    return historic_price



price_data('024850')

In [ ]:
import pandas as pd
import requests

def make_fs_dataframe(firm_code):
    fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode='+firm_code
    fs_page=requests.get(fs_url)
    fs_tables=pd.read_html(fs_page.text)
    
    temp_df=fs_tables[0]
    temp_df=temp_df.set_index(temp_df.columns[0])
    temp_df=temp_df[temp_df.columns[:4]]
    temp_df=temp_df.loc[['매출액','영업이익','당기순이익']]
    
    temp_df2=fs_tables[2]
    temp_df2=temp_df2.set_index(temp_df2.columns[0])
    temp_df2=temp_df2.loc[['자산','부채','자본']]
    
    temp_df3=fs_tables[4]
    temp_df3=temp_df3.set_index(temp_df3.columns[0])
    temp_df3=temp_df3.loc[['영업활동으로인한현금흐름']]
    
    fs_df=pd.concat([temp_df, temp_df2, temp_df3])
    return fs_df

def changed_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code: dataframe[col]})
        temp_df=temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]
        if num==0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how="outer", left_index=True, right_index=True)
    return total_df

#fs 지표 firmcode 별로 만들기
def fs_total(firmcode_list):
    for num, code in enumerate(firmcode_list):
        fs_df = make_fs_dataframe(code)
        fs_df_changed=changed_df(code, fs_df)
        if num == 0:
            total_fs = fs_df_changed
        else:
            total_fs=pd.concat([total_fs, fs_df_changed])
    return total_fs



def make_fr_dataframe(firm_code):
    fr_url='http://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701&gicode='+firm_code
    fr_page = requests.get(fr_url)
    fr_tables=pd.read_html(fr_page.text)
    temp_df = fr_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df=temp_df.loc[['ROA계산에 참여한 계정 펼치기','유동비율계산에 참여한 계정 펼치기', '부채비율계산에 참여한 계정 펼치기', 'ROIC계산에 참여한 계정 펼치기']]
    temp_df.index=['ROA','유동비율', '부채비율', 'ROIC']
    return temp_df

#fr 지표 firmcode 별로 만들기
def fr_total(firmcode_list):
    for num, code in enumerate(firmcode_list):
        fr_df=make_fr_dataframe(code)
        fr_df_changed = changed_df(code, fr_df)
        if num ==0 :
            total_fr=fr_df_changed
        else:
            total_fr = pd.concat([total_fr, fr_df_changed])
    return total_fr



def make_ir_dataframe(firm_code):
    ir_url='http://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701&gicode='+firm_code
    ir_page = requests.get(ir_url)
    ir_tables=pd.read_html(ir_page.text)
    temp_df=ir_tables[1]
    temp_df=temp_df.set_index(temp_df.columns[0])
    temp_df=temp_df.loc[['PER계산에 참여한 계정 펼치기', 'PCR계산에 참여한 계정 펼치기', 'PBR계산에 참여한 계정 펼치기', 'PSR계산에 참여한 계정 펼치기', '총현금흐름']]
    temp_df.index=['PER', 'PCR', 'PBR', 'PSR', '총현금흐름']
    return temp_df

def ir_total(firmcode_list):
    for num, code in enumerate(firmcode_list):
        ir_df=make_ir_dataframe(code)
        ir_df_changed = changed_df(code, ir_df)
        if num ==0 :
            total_ir=ir_df_changed
        else:
            total_ir = pd.concat([total_ir, ir_df_changed])
    return total_ir


path=r'/Users/donghyunyoo/Downloads/firm_code_list.xls'
code_data = pd.read_excel(path)
code_data= code_data[['종목코드','기업명']]


#fnguide에서 다운받은 firm 엑셀 리스트를 리스토로 만들기
def make_code(x):
    x=str(x)
    return 'A'+'0'*(6-len(x))+x

code_data['종목코드']=code_data['종목코드'].apply(make_code)

#firm_list=code_data['종목코드'].values.tolist()

#전종목 데이터가져오기
import time

for num, code in enumerate(code_data['종목코드']):
    try:
        print(num,code)
        time.sleep(0.01)
        try:
            fss_df=make_fs_dataframe(code)
        except requests.exeptions.Timeout:
            time.sleep(2)
            fss_df=make_fs_dataframe(code)
        fss_df_changed=change_df(code, fs_df)
        if num ==0:
            total_fss=fss_df_changed
        else:
            total_fss=pd.concat([total_fss,fss_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue
        
total_fss.to_excel(r'/Users/donghyunyoo/Downloads/total_fss.xls')